# Binary classification with Keras neural network

We'll be using this notebook to train our model to detect liquor bottles in the lab.

### Imports

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

import os
import numpy as np
import pandas as np

import matplotlib.pyplot as plt
%matplotlib inline

### Verify our directories structure

In [6]:
print(os.listdir("./"))

print(os.listdir("./liquor"))

print(os.listdir("./liquor/test"))

print(os.listdir("./liquor/train/"))

print(os.listdir("./liquor/val/"))

### Check an image in the "HENDRICKS" training set

In [8]:
img_name = 'HENDRICKS0012.jpg'
img_hendricks = load_img('./liquor/train/HENDRICKS/' + img_name)

print('HENDRICKS')
plt.imshow(img_hendricks)
plt.show()

### Check an image in the HERRADURA training set

In [10]:
img_name = 'HERRADURA0040.jpg'
img_herradura = load_img('./liquor/train/HERRADURA/' + img_name)

print('HERRADURA')
plt.imshow(img_herradura)
plt.show()

### Initialize variables

In [13]:
# dimensions of our images.
img_width, img_height = 50, 50

In [14]:
train_data_dir = './liquor/train'
validation_data_dir = './liquor/val'
test_data_dir = './liquor/test'

nb_train_samples = 512
nb_validation_samples = 17
epochs = 20
batch_size = 16

In [15]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

### Create Sequential model

In [17]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

### Check information about the model

Display some information about the model we're using.

In [20]:
model.summary()

In [21]:
model.input

In [22]:
model.output

### Compile the model

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### Upload images from the different sets

In [26]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [27]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [28]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [29]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [30]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

### Fit the model

In [32]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

### Save the model (weights + complete model)

In [34]:
model.save_weights('liquor_first_try.h5')

In [35]:
model.save('liquor_model.h5')

### Evaluate the model

In [37]:
# evaluate the model
scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))